In [11]:
from typing_extensions import Self
import pandas as pd
from typing import List, Dict
import numpy as np

class INSEEDataProcessor:
    """
    Classe pour traiter les données INSEE de démographie et économie.
    Permet de charger, filtrer et renommer les colonnes des données communales.
    Gère automatiquement la conversion des décimales françaises (virgule -> point).
    """

    def __init__(self, bucket_path: str):
        """
        Initialise le processeur avec le chemin vers les données.

        Args:
            bucket_path (str): Chemin vers le fichier CSV (local ou cloud)
        """
        self.bucket_path = bucket_path
        self.df_raw = None
        self.df_processed = None

        # Configuration des colonnes à conserver et leurs nouveaux noms
        self._setup_column_mapping()

    def _setup_column_mapping(self) -> None:
        """Configure le mapping entre les codes INSEE et les noms explicites."""

        # Colonnes originales à conserver (codes INSEE)
        self.colonnes_insee = [
            "CODGEO",                    # Code géographique commune
            "MED21",                     # Médiane du niveau de vie 2021
            "ETTOT22",                   # Entreprises actives 2022
            "ENCTOT17",                  # Créations d'entreprises 2017
            "ENCTOT22",                  # Créations d'entreprises 2022
            "P21_RP",                    # Résidences principales totales
            "P21_RP_PROP",              # Résidences principales propriétaires
            "P21_RP_LOC",               # Résidences principales locataires
            "C21_POP55P",               # Population 55 ans et plus
            "P21_POP1824",               # Population de 18-24 ans (à aggréger à 25-29)
            "P21_POP2529",               # Population de 24-29 ans (à aggréger à 18-24)
            "P21_ETUD1564",             # Étudiants 15-64 ans
            "P21_NPER_RP_LOCHLMV",      # Logements HLM
            "P21_POP",                  # Population totale
            "SUPERF",                   # Superficie en km²
            "P21_ACTOCC15P_VELO",       # Actifs utilisant le vélo
            "P21_ACTOCC15P_VOITURE",    # Actifs utilisant la voiture
            "P21_ACTOCC15P_COMMUN",     # Actifs utilisant transports en commun
            "P21_CHOM1564",             # Chômeurs 15-64 ans
            "P21_ACT1564",              # Actifs 15-64 ans
            "C21_EMPLT",                # Total d'emlois par secteur
            "C21_EMPLT_AGRI",           # Emplois agriculture
            "C21_EMPLT_INDUS",          # Emplois industrie
            "C21_EMPLT_CONST",          # Emplois construction
            "C21_EMPLT_CTS",            # Emplois commerce/transport/services
            "C21_EMPLT_APESAS",         # Emplois administration publique
            "P21_NSCOL15P",             # Non scolarisés 15 ans et plus
            "P21_NSCOL15P_DIPLMIN",     # Sans diplôme
            "P21_NSCOL15P_BEPC",        # Diplôme brevet
            "P21_NSCOL15P_CAPBEP",      # CAP/BEP
            "P21_NSCOL15P_BAC",         # Baccalauréat
            "P21_NSCOL15P_SUP2",        # Bac+2
            "P21_NSCOL15P_SUP34",       # Bac+3/4
            "P21_NSCOL15P_SUP5"         # Bac+5 et plus
        ]

        # Noms explicites correspondants
        self.noms_explicites = [
            "code_geo",
            "mediane_niveau_vie",
            "entreprises_actives_2022",
            "entreprises_creations_2017",
            "entreprises_creations_2022",
            "base_taux_rp_prop_loc_pop",
            "taux_proprietaires_pop",
            "taux_locataires_pop",
            "taux_55_plus_pop",
            "taux_18_29_pop",
            "taux_18_29_bis",
            "taux_etudiants_pop",
            "taux_hlm_pop",
            "population",
            "superficie_km2",
            "taux_actifs_velo",
            "taux_actifs_voiture",
            "taux_actifs_transports_commun",
            "taux_chomage",
            "base_taux_actifs",
            "base_total_emploi",
            "taux_agri_emploi",
            "taux_indus_emploi",
            "taux_constr_emploi",
            "taux_tertiaire_emploi",
            "taux_public_emploi",
            "pop15_non_sco",
            "taux_sans_diplome_pop15ns",
            "taux_brevet_pop15ns",
            "taux_cap_pop15ns",
            "taux_bac_pop15ns",
            "taux_bts_pop15ns",
            "taux_licence_pop15ns",
            "taux_master_doctorat_pop15ns"
        ]

        # Vérification de la cohérence
        if len(self.colonnes_insee) != len(self.noms_explicites):
            raise ValueError("Le nombre de colonnes INSEE et de noms explicites doit être identique")

    def load_data(self, sep: str = ',') -> pd.DataFrame:
        """
        Charge les données depuis le fichier CSV et convertit les décimales françaises.

        Args:
            sep (str): Séparateur utilisé dans le CSV

        Returns:
            pd.DataFrame: DataFrame brut chargé avec décimales converties
        """
        try:
            print(f"Chargement des données depuis : {self.bucket_path}")

            # Chargement initial en gardant tout en string pour la conversion
            self.df_raw = pd.read_csv(self.bucket_path, sep=sep, dtype=str, keep_default_na=False)
            print(f"Données chargées avec succès : {self.df_raw.shape[0]} lignes, {self.df_raw.shape[1]} colonnes")

            # Conversion des décimales françaises intégrée
            print("Conversion des décimales françaises...")
            conversions_effectuees = 0

            # Parcourir toutes les colonnes sauf CODGEO (qui doit rester en string)
            colonnes_a_convertir = [col for col in self.df_raw.columns if col != 'CODGEO']

            for col in colonnes_a_convertir:
                # Vérifier s'il y a des virgules dans les données
                has_comma = self.df_raw[col].astype(str).str.contains(',', na=False).any()

                if has_comma:
                    # Remplacer les virgules par des points
                    self.df_raw[col] = self.df_raw[col].astype(str).str.replace(',', '.', regex=False)
                    conversions_effectuees += 1
                    print(f"  - Colonne '{col}': virgules converties en points")

                # Essayer de convertir en numérique
                try:
                    # Remplacer les valeurs non numériques comme 's' par NaN
                    self.df_raw[col] = self.df_raw[col].replace(['s', 'S', '', ' '], pd.NA)
                    self.df_raw[col] = pd.to_numeric(self.df_raw[col], errors='coerce')
                except:
                    # Si la conversion échoue, on garde la colonne comme elle est
                    pass

            print(f"Conversion terminée : {conversions_effectuees} colonnes traitées")

            return self.df_raw
        except Exception as e:
            raise Exception(f"Erreur lors du chargement des données : {str(e)}")

    def filter_columns(self) -> pd.DataFrame:
        """
        Filtre le DataFrame pour ne conserver que les colonnes d'intérêt.

        Returns:
            pd.DataFrame: DataFrame filtré
        """
        if self.df_raw is None:
            raise ValueError("Les données doivent être chargées avant le filtrage")

        # Vérification de la présence des colonnes
        colonnes_manquantes = [col for col in self.colonnes_insee if col not in self.df_raw.columns]
        if colonnes_manquantes:
            print(f"Attention : colonnes manquantes : {colonnes_manquantes}")

        # Filtrage avec gestion des colonnes manquantes
        colonnes_disponibles = [col for col in self.colonnes_insee if col in self.df_raw.columns]
        self.df_processed = self.df_raw[colonnes_disponibles].copy()

        print(f"Filtrage effectué : {len(colonnes_disponibles)} colonnes conservées")
        return self.df_processed

    def rename_columns(self) -> pd.DataFrame:
        """
        Renomme les colonnes avec des noms plus explicites.

        Returns:
            pd.DataFrame: DataFrame avec colonnes renommées
        """
        if self.df_processed is None:
            raise ValueError("Les données doivent être filtrées avant le renommage")

        # Création du dictionnaire de mapping pour les colonnes disponibles
        colonnes_disponibles = list(self.df_processed.columns)
        mapping = {}

        for i, col_insee in enumerate(self.colonnes_insee):
            if col_insee in colonnes_disponibles:
                mapping[col_insee] = self.noms_explicites[i]

        # Renommage
        self.df_processed.rename(columns=mapping, inplace=True)
        print(f"Renommage effectué : {len(mapping)} colonnes renommées")

        return self.df_processed

    def handle_missing_mediane_niveau_vie(self) -> pd.DataFrame:
        """
        Traite les valeurs manquantes dans la colonne 'mediane_niveau_vie'.

        Returns:
            pd.DataFrame: DataFrame avec valeurs manquantes gérées.
        """
        if self.df_processed is None:
            raise ValueError("Les données doivent être chargées et renommées avant ce traitement.")

        if "mediane_niveau_vie" not in self.df_processed.columns:
            print("La colonne 'mediane_niveau_vie' n'existe pas, rien à traiter.")
            return self.df_processed

        # La conversion a déjà été faite dans load_data, on vérifie juste les valeurs manquantes
        n_missing = self.df_processed["mediane_niveau_vie"].isna().sum()
        if n_missing == 0:
            print("Aucune valeur manquante dans 'mediane_niveau_vie'.")
            return self.df_processed

        print(f"Traitement de {n_missing} valeurs manquantes dans 'mediane_niveau_vie'...")
        try:
            median_value = self.df_processed["mediane_niveau_vie"].median()
            self.df_processed["mediane_niveau_vie"] = self.df_processed["mediane_niveau_vie"].fillna(median_value)
            print(f"Valeurs remplacées par la médiane : {median_value}")
        except Exception as e:
            print(f"Erreur lors du calcul/remplacement de la médiane : {e}")
        return self.df_processed

    def calculate_variation_entreprises_open(self) -> pd.DataFrame:
        """
        Calcule la variation relative et absolue du nombre de créations d'entreprises entre 2017 et 2022.
        Ajoute deux colonnes : variation_absolue_creations_2017_2022 et variation_pourcent_creations_2017_2022 (%)

        Returns:
            pd.DataFrame: DataFrame avec la variation calculée
        """
        # Les colonnes sont déjà converties en numérique grâce à _convert_french_decimals
        print("Calcul de la variation des créations d'entreprises...")

        # S'assurer que les colonnes sont bien numériques
        self.df_processed["entreprises_creations_2017"] = pd.to_numeric(self.df_processed["entreprises_creations_2017"], errors="coerce")
        self.df_processed["entreprises_creations_2022"] = pd.to_numeric(self.df_processed["entreprises_creations_2022"], errors="coerce")

        # Variation relative en pourcentage selon formule mathématique
        self.df_processed["entreprises_variation_creations_2017_2022"] = ((
            (self.df_processed["entreprises_creations_2022"] - self.df_processed["entreprises_creations_2017"])
            / self.df_processed["entreprises_creations_2017"]) * 100
        ).round(2)

        # Remplacer les valeurs infinies ou NaN par 0
        self.df_processed["entreprises_variation_creations_2017_2022"] = self.df_processed["entreprises_variation_creations_2017_2022"].fillna(0)
        self.df_processed["entreprises_variation_creations_2017_2022"] = self.df_processed["entreprises_variation_creations_2017_2022"].replace([float('inf'), float('-inf')], 0)

        print("Variation des créations d'entreprises calculée")
        self.df_processed = self.df_processed.drop(columns=['entreprises_creations_2017'])
        self.df_processed = self.df_processed.drop(columns=['entreprises_creations_2022'])
        return self.df_processed

    def calculate_percentages(self) -> pd.DataFrame:
        """
        Calcule les pourcentages pour les différents indicateurs.

        Returns:
            pd.DataFrame: DataFrame avec les pourcentages calculés
        """
        if self.df_processed is None:
            raise ValueError("Les données doivent être traitées avant le calcul des pourcentages")

        print("Calcul des pourcentages...")

        # Configuration des calculs de pourcentages
        calculs_config = {
            # Pourcentages basés sur la population totale
            'population': [
                'taux_55_plus_pop',
                'taux_etudiants_pop',
                "taux_18_29_pop",
                "taux_18_29_bis"
            ],

            # Pourcentages basés sur les résidences principales (P21_RP)
            'base_taux_rp_prop_loc_pop': [
                'taux_proprietaires_pop',
                'taux_locataires_pop',
                'taux_hlm_pop'
            ],

            # Pourcentages basés sur les actifs (taux_actifs)
            'base_taux_actifs': [
                'taux_actifs_velo',
                'taux_actifs_voiture',
                'taux_actifs_transports_commun'
            ],

            # Pourcentages basés sur le total d'emplois par secteur (taux_total_emploi)
            'base_total_emploi': [
                'taux_agri_emploi',
                'taux_indus_emploi',
                'taux_constr_emploi',
                'taux_tertiaire_emploi',
                'taux_public_emploi'
            ],

            # Pourcentages basés sur la population 15+ non scolarisée
            'pop15_non_sco': [
                'taux_chomage',
                'taux_sans_diplome_pop15ns',
                'taux_brevet_pop15ns',
                'taux_cap_pop15ns',
                'taux_bac_pop15ns',
                'taux_bts_pop15ns',
                'taux_licence_pop15ns',
                'taux_master_doctorat_pop15ns'
            ]
        }

        # Calcul des pourcentages
        for base_colonne, colonnes_a_calculer in calculs_config.items():
            if base_colonne in self.df_processed.columns:
                for colonne in colonnes_a_calculer:
                    if colonne in self.df_processed.columns:
                        # S'assurer que les colonnes sont numériques
                        self.df_processed[base_colonne] = pd.to_numeric(self.df_processed[base_colonne], errors="coerce")
                        self.df_processed[colonne] = pd.to_numeric(self.df_processed[colonne], errors="coerce")

                        # Calcul du pourcentage avec gestion des divisions par zéro
                        self.df_processed[colonne] = (
                            self.df_processed[colonne] / self.df_processed[base_colonne] * 100
                        ).round(2)

                        # Remplacer les valeurs infinies ou NaN par 0
                        self.df_processed[colonne] = self.df_processed[colonne].fillna(0)
                        self.df_processed[colonne] = self.df_processed[colonne].replace([float('inf'), float('-inf')], 0)

        # Calcul spécial : densité de population (hab/km²)
        if 'superficie_km2' in self.df_processed.columns and 'population' in self.df_processed.columns:
            # S'assurer que les colonnes sont numériques
            self.df_processed['superficie_km2'] = pd.to_numeric(self.df_processed['superficie_km2'], errors="coerce")
            self.df_processed['population'] = pd.to_numeric(self.df_processed['population'], errors="coerce")

            self.df_processed['densite_population'] = (
                self.df_processed['population'] / self.df_processed['superficie_km2']
            ).round(2)

            # Gestion des divisions par zéro
            self.df_processed['densite_population'] = self.df_processed['densite_population'].fillna(0)
            self.df_processed['densite_population'] = self.df_processed['densite_population'].replace([float('inf'), float('-inf')], 0)

            print("Densité de population calculée (hab/km²)")

        print("Calculs de pourcentages terminés")
        print("Note : Les taux de propriétaires et locataires sont maintenant calculés par rapport aux résidences principales")

        return self.df_processed

    def filter_percentage_columns(self) -> pd.DataFrame:
        """
        Filtre le DataFrame pour ne conserver que code_geo et les colonnes calculées en pourcentages.

        Returns:
            pd.DataFrame: DataFrame avec seulement les colonnes de pourcentages
        """
        if self.df_processed is None:
            raise ValueError("Les données doivent être traitées avant le filtrage des pourcentages")

        # Combinaison des colonnes 18-29 ans
        if 'taux_18_29_pop' in self.df_processed.columns and 'taux_18_29_bis' in self.df_processed.columns:
            self.df_processed['taux_18_29_pop'] = self.df_processed['taux_18_29_pop'].fillna(0) + \
                                                  self.df_processed['taux_18_29_bis'].fillna(0)
        # Suppression de la colonne bis
        self.df_processed = self.df_processed.drop(columns=['taux_18_29_bis'])

        # Combinaison industrie + construction = secteur secondaire
        colonnes_secondaire = ['taux_indus_emploi', 'taux_constr_emploi']
        if all(col in self.df_processed.columns for col in colonnes_secondaire):
            self.df_processed['taux_secondaire_emploi'] = (
                self.df_processed['taux_indus_emploi'].fillna(0) +
                self.df_processed['taux_constr_emploi'].fillna(0)
            )
            # Suppression des colonnes individuelles
            self.df_processed = self.df_processed.drop(columns=colonnes_secondaire)

        # Combinaison tertiaire + public
        colonnes_tertiaire_public = ['taux_tertiaire_emploi', 'taux_public_emploi']
        if all(col in self.df_processed.columns for col in colonnes_tertiaire_public):
            self.df_processed['taux_tertiaire_emploi'] = (
                self.df_processed['taux_tertiaire_emploi'].fillna(0) +
                self.df_processed['taux_public_emploi'].fillna(0)
            )
            # Suppression de la colonne publique
            self.df_processed = self.df_processed.drop(columns=['taux_public_emploi'])

        # Identification des colonnes de pourcentages (celles qui commencent par 'taux_' ou 'entreprises_')
        colonnes_pourcentages = [col for col in self.df_processed.columns if col.startswith('taux_') or col.startswith('entreprises_')]

        # Ajout de la densité de population si elle existe
        if 'densite_population' in self.df_processed.columns:
            colonnes_pourcentages.append('densite_population')

        # Ajout de la médiane du niveau de vie (en €) si elle existe
        if 'mediane_niveau_vie' in self.df_processed.columns:
            colonnes_pourcentages.append('mediane_niveau_vie')

        # Colonnes finales à conserver : code_geo + colonnes de pourcentages
        colonnes_finales = ['code_geo'] + colonnes_pourcentages

        # Vérification que code_geo existe
        if 'code_geo' not in self.df_processed.columns:
            raise ValueError("La colonne 'code_geo' n'existe pas dans les données traitées")

        # Filtrage final
        self.df_processed = self.df_processed[colonnes_finales].copy()

        print(f"Filtrage final effectué : {len(colonnes_finales)} colonnes conservées")
        print(f"Colonnes conservées : {list(self.df_processed.columns)}")

        return self.df_processed

    def process_data(self, sep: str = ',') -> pd.DataFrame:
        """
        Pipeline complet de traitement des données.

        Args:
            sep (str): Séparateur du fichier CSV

        Returns:
            pd.DataFrame: DataFrame final traité avec seulement les pourcentages
        """
        print("=== Début du traitement des données INSEE ===")

        # Étapes du pipeline
        self.load_data(sep=sep)           # chargement de la data avec conversion des décimales
        self.filter_columns()             # étape filtrer parmi les 2000 colonnes ce qui nous intéresse pour nos petits calculs
        self.rename_columns()             # étape pour renommer les colonnes
        self.handle_missing_mediane_niveau_vie() # pour gérer les 4 communes de - 100 habitants sans médiane des revenus
        self.calculate_variation_entreprises_open() # pour calculer la variation entre le nombre de créations d'entreprise entre 2017 et 2022
        self.calculate_percentages()      # étape pour calculer les pourcentages
        self.filter_percentage_columns()  #  étape pour filtrer les pourcentages uniquement

        print("=== Traitement terminé ===")
        return self.df_processed

    def get_summary(self) -> Dict:
        """
        Retourne un résumé des données traitées.

        Returns:
            Dict: Statistiques sur les données
        """
        if self.df_processed is None:
            return {"error": "Aucune donnée traitée"}

        # Calcul des statistiques par type de colonne
        colonnes_pourcentages = [col for col in self.df_processed.columns if col.startswith('taux_') or col.startswith('entreprises_')]
        colonnes_absolues = [col for col in self.df_processed.columns
                           if not (col.startswith('taux_') or col.startswith('entreprises_'))
                           and col not in ['code_geo', 'mediane_niveau_vie', 'densite_population']]

        return {
            "nb_lignes": len(self.df_processed),
            "nb_colonnes": len(self.df_processed.columns),
            "colonnes_pourcentages": len(colonnes_pourcentages),
            "colonnes_valeurs_absolues": len(colonnes_absolues),
            "valeurs_manquantes": self.df_processed.isnull().sum().sum(),
            "taille_memoire_mb": round(self.df_processed.memory_usage(deep=True).sum() / 1024**2, 2),
            "densite_moyenne": round(self.df_processed['densite_population'].mean(), 2) if 'densite_population' in self.df_processed.columns else "Non calculée"
        }

    def get_percentage_validation(self) -> Dict:
        """
        Valide la cohérence des pourcentages calculés.

        Returns:
            Dict: Rapport de validation des pourcentages
        """
        if self.df_processed is None:
            return {"error": "Aucune donnée traitée"}

        validation = {
            "pourcentages_negatifs": {},
            "pourcentages_superieurs_100": {},
            "valeurs_nulles": {},
            "statistiques_par_groupe": {}
        }

        # Groupes de validation
        groupes_validation = {
            "Population": ["taux_55_plus_pop", "taux_etudiants_pop",
                          "taux_hlm_pop"],
            "Résidences_Principales": ["taux_proprietaires_pop", "taux_locataires_pop"],
            "Actifs": ["taux_actifs_velo", "taux_actifs_voiture",
                      "taux_actifs_transports_commun", "taux_chomage"],
            "Emploi_Par_Secteur": ["taux_agri_emploi", "taux_indus_emploi",
                      "taux_constr_emploi", "taux_tertiaire_emploi",
                      "taux_public_emploi"],
            "Diplômes": ["taux_sans_diplome_pop15ns", "taux_brevet_pop15ns",
                        "taux_cap_pop15ns", "taux_bac_pop15ns",
                        "taux_bts_pop15ns", "taux_licence_pop15ns",
                        "taux_master_doctorat_pop15ns"]
        }

        for groupe, colonnes in groupes_validation.items():
            colonnes_existantes = [col for col in colonnes if col in self.df_processed.columns]

            if colonnes_existantes:
                df_groupe = self.df_processed[colonnes_existantes]

                # Détection des anomalies
                for col in colonnes_existantes:
                    negatifs = (df_groupe[col] < 0).sum()
                    superieurs_100 = (df_groupe[col] > 100).sum()
                    nulles = (df_groupe[col] == 0).sum()

                    if negatifs > 0:
                        validation["pourcentages_negatifs"][col] = negatifs
                    if superieurs_100 > 0:
                        validation["pourcentages_superieurs_100"][col] = superieurs_100
                    if nulles > 0:
                        validation["valeurs_nulles"][col] = nulles

                # Statistiques du groupe
                validation["statistiques_par_groupe"][groupe] = {
                    "moyenne": df_groupe.mean().round(2).to_dict(),
                    "mediane": df_groupe.median().round(2).to_dict(),
                    "max": df_groupe.max().round(2).to_dict(),
                    "min": df_groupe.min().round(2).to_dict()
                }

        return validation

def main():
    """Fonction principale pour exécuter le traitement."""

    # Configuration
    bucket_path = 'gs://mspr-data-ia-raw/idf_INSEE_dossier_complet.csv'

    # Initialisation du processeur
    processor = INSEEDataProcessor(bucket_path)

    # Traitement des données
    df_final = processor.process_data()

    # Affichage des résultats
    print("\n=== Aperçu des données ===")
    display(df_final.head())

    print("\n=== Résumé ===")
    summary = processor.get_summary()
    for key, value in summary.items():
        print(f"{key}: {value}")

    print("\n=== Validation ===")
    validation = processor.get_percentage_validation()
    for key, value in validation.items():
        print(f"{key}: {value}")

    return df_final

# Exécution si le script est lancé directement
if __name__ == "__main__":
    df_traite = main()

=== Début du traitement des données INSEE ===
Chargement des données depuis : gs://mspr-data-ia-raw/idf_INSEE_dossier_complet.csv
Données chargées avec succès : 1266 lignes, 1927 colonnes
Conversion des décimales françaises...
  - Colonne 'PIMP21': virgules converties en points
  - Colonne 'TP6021': virgules converties en points
  - Colonne 'TP60AGE121': virgules converties en points
  - Colonne 'TP60AGE221': virgules converties en points
  - Colonne 'TP60AGE321': virgules converties en points
  - Colonne 'TP60AGE421': virgules converties en points
  - Colonne 'TP60AGE521': virgules converties en points
  - Colonne 'TP60AGE621': virgules converties en points
  - Colonne 'TP60TOL121': virgules converties en points
  - Colonne 'TP60TOL221': virgules converties en points
  - Colonne 'PACT21': virgules converties en points
  - Colonne 'PTSA21': virgules converties en points
  - Colonne 'PCHO21': virgules converties en points
  - Colonne 'PBEN21': virgules converties en points
  - Colonne '

,code_geo,entreprises_actives_2022,taux_proprietaires_pop,taux_locataires_pop,taux_55_plus_pop,taux_18_29_pop,taux_etudiants_pop,taux_hlm_pop,taux_actifs_velo,taux_actifs_voiture,...,taux_brevet_pop15ns,taux_cap_pop15ns,taux_bac_pop15ns,taux_bts_pop15ns,taux_licence_pop15ns,taux_master_doctorat_pop15ns,entreprises_variation_creations_2017_2022,taux_secondaire_emploi,densite_population,mediane_niveau_vie
0,77001,38.0,93.27,3.37,42.06,9.43,6.29,0.0,0.34,78.15,...,5.28,18.55,17.65,15.77,14.44,17.03,84.62,11.33,91.59,30970.0
1,77002,31.0,82.14,14.16,37.85,8.59,5.19,0.0,0.27,78.31,...,6.33,27.73,18.99,9.62,5.88,8.01,-28.57,21.52,40.43,26310.0
2,77003,8.0,86.20,12.48,43.04,9.38,3.68,0.0,0.00,80.85,...,4.77,23.83,22.69,13.90,9.52,9.88,0.00,42.16,28.13,28400.0
3,77004,15.0,83.87,11.50,27.09,8.14,1.82,0.0,0.00,79.29,...,4.85,26.03,22.88,8.94,11.62,8.23,-57.14,45.15,38.49,28400.0
4,77005,74.0,78.42,20.13,29.78,13.06,7.94,0.0,0.64,78.22,...,5.45,22.04,20.72,14.17,14.59,10.40,42.86,22.39,251.48,29060.0



=== Résumé ===
nb_lignes: 1266
nb_colonnes: 25
colonnes_pourcentages: 22
colonnes_valeurs_absolues: 0
valeurs_manquantes: 0
taille_memoire_mb: 0.31
densite_moyenne: 1445.39

=== Validation ===
pourcentages_negatifs: {}
pourcentages_superieurs_100: {'taux_hlm_pop': np.int64(39)}
valeurs_nulles: {'taux_hlm_pop': np.int64(497), 'taux_proprietaires_pop': np.int64(1), 'taux_locataires_pop': np.int64(1), 'taux_actifs_velo': np.int64(289), 'taux_actifs_transports_commun': np.int64(4), 'taux_chomage': np.int64(2), 'taux_agri_emploi': np.int64(500), 'taux_tertiaire_emploi': np.int64(22), 'taux_brevet_pop15ns': np.int64(2), 'taux_licence_pop15ns': np.int64(3), 'taux_master_doctorat_pop15ns': np.int64(3)}
statistiques_par_groupe: {'Population': {'moyenne': {'taux_55_plus_pop': 30.05, 'taux_etudiants_pop': 6.46, 'taux_hlm_pop': 20.96}, 'mediane': {'taux_55_plus_pop': 29.24, 'taux_etudiants_pop': 6.42, 'taux_hlm_pop': 2.99}, 'max': {'taux_55_plus_pop': 76.09, 'taux_etudiants_pop': 30.35, 'taux_hlm

In [ ]:
# Sauvegarde locale temporaire
df_traite.to_csv('test22.csv', index=False)

In [12]:
from pandas_gbq import to_gbq

def export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id):
    """
    Exporte un DataFrame vers BigQuery avec pandas-gbq

    Args:
        df: DataFrame à exporter
        project_id: ID du projet GCP
        dataset_id: ID du dataset BigQuery
        table_id: ID de la table BigQuery
    """

    # Configuration de la destination
    destination_table = f"{dataset_id}.{table_id}"

    try:
        # Export vers BigQuery
        to_gbq(
            df,
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Options: 'fail', 'replace', 'append'
            progress_bar=True,
            auth_local_webserver=False
        )

        print(f"✅ DataFrame exporté avec succès vers {project_id}.{destination_table}")
        print(f"📊 Nombre de lignes exportées : {len(df)}")
        print(f"📋 Nombre de colonnes : {len(df.columns)}")

    except Exception as e:
        print(f"❌ Erreur lors de l'export : {str(e)}")

df = df_traite
project_id = "mspr-data-ia"
dataset_id = "idf_silver_cleaned"
table_id = "dim_contexte_socio_economique"
export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id)

100%|██████████| 1/1 [00:00<00:00, 7108.99it/s]

✅ DataFrame exporté avec succès vers mspr-data-ia.idf_silver_cleaned.dim_contexte_socio_economique
📊 Nombre de lignes exportées : 1266
📋 Nombre de colonnes : 25
